instalar bibliotecas

ler a base de dados 

definir preço máximo e preço mínimo

pesquisar no google:
- abrir o navegador
- pesquisar pelo produto
- selecionar os produtos que estão dentro da faixa de preço
- armazenar: nome, preço e link
- filtrar os termos banidos

pesquisar no buscapé:
- abrir o navegador
- pesquisar pelo produto
- selecionar os produtos que estão dentro da faixa de preço
- armazenar: nome, preço e link
- filtrar os termos banidos

### Importar bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import pandas as pd

options = Options()
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36")

servico = Service(ChromeDriverManager().install())
navegador = webdriver.Chrome(service=servico, options=options)

# importar/visualizar a base de dados
tabela_produtos = pd.read_excel('buscas.xlsx')
print(tabela_produtos)

              Nome Termos banidos  Preço mínimo  Preço máximo
0  iphone 13 128gb     mini watch          2500          3500
1         rtx 3060     zota galax          1200          4500


### Definição das funções de busca no google e no buscape

In [2]:
def busca_google_shopping(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    #entrar no google
    navegador.get('https://www.google.com/')
    time.sleep(3)

    #pesquisar pelo produto
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_nome_produto = produto.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    #pesquisar o nome do produto no Google
    navegador.find_element(By.CLASS_NAME, 'gLFyf').send_keys(produto + Keys.ENTER)
    time.sleep(3)

    #clicar na aba shopping
    elementos = navegador.find_elements(By.CLASS_NAME, 'R1QWuf')

    for item in elementos:
        if "Shopping" in item.text:
            item.click()
            break

    time.sleep(3)

    print('busca realizada')

    #pegar as informações do produtos
    lista_elementos = navegador.find_elements('class name', 'pla-unit-container')

    lista_ofertas = [] # lista que a função vai me dar como resposta

    for elemento in lista_elementos:
        nome = elemento.find_element(By.CSS_SELECTOR, '[role="heading"]').text
        nome = nome.lower()

        # analisar se ele não tem termos banidos
        tem_termo_banido = False
        for termo in lista_termos_banidos:
            if termo in nome:
                tem_termo_banido = True

        # analisar se ele tem todos os termos do nome do produto
        tem_todos_termos = True
        for termo in lista_termos_nome_produto:
            if termo not in nome:
                tem_todos_termos = False

        if not tem_termo_banido and tem_todos_termos: #verificando o nome
            try:
                #print('nome tá dentro dos termos', nome)
                preco = elemento.find_element(By.CSS_SELECTOR, '.VbBaOe').text
                preco = preco.replace('R$', '').replace('.','').replace(' ','').replace(',','.')
                preco = float(preco)
                #verificando se o preço está dentro do intervalo
                if preco_minimo <= preco <= preco_maximo:
                    print('{} está dentro do intervalo. Preço: {}'.format(nome, preco))
                    link = elemento.find_element(By.CSS_SELECTOR, 'a.plantl').get_attribute('href')
                    lista_ofertas.append((nome, preco, link))
            except:
                continue
    return(lista_ofertas)


def busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
    #entrar no Buscapé
    navegador.get('https://www.buscape.com.br/')
    time.sleep(3)

    #pesquisar pelo produto
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_nome_produto = produto.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    #pesquisar o nome do produto no buscapé
    navegador.find_element(By.CLASS_NAME, 'SearchInput_Input__W2vzU').send_keys(produto + Keys.ENTER)
    time.sleep(3)

    #pegar as informações do produtos
    lista_elementos = navegador.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')
    #print(lista_elementos)
    lista_ofertas = [] # lista que a função vai me dar como resposta

    for elemento in lista_elementos:
        nome = elemento.find_element(By.CSS_SELECTOR, ".ProductCard_ProductCard_Name__U_mUQ").text
        nome = nome.lower()

        # analisar se ele não tem termos banidos
        tem_termo_banido = False
        for termo in lista_termos_banidos:
            if termo in nome:
                tem_termo_banido = True

        # analisar se ele tem todos os termos do nome do produto
        tem_todos_termos = True
        for termo in lista_termos_nome_produto:
            if termo not in nome:
                tem_todos_termos = False
        
        if not tem_termo_banido and tem_todos_termos: #verificando o nome
            try:
                #print('nome tá dentro dos termos', nome)
                preco = elemento.find_element(By.CSS_SELECTOR, '.Text_Text__ARJdp.Text_MobileHeadingS__HEz7L').text
                preco = preco.replace('R$', '').replace('.','').replace(' ','').replace(',','.')
                preco = float(preco)
        
                #verificando se o preço está dentro do intervalo
                if preco_minimo <= preco <= preco_maximo:
                    #print('{} está dentro do intervalo. Preço: {}'.format(nome, preco))
                    try:
                        anchors = elemento.find_elements(By.CSS_SELECTOR, 'a[href]')
                        if anchors:#
                            link = anchors[0].get_attribute('href')
                        else:
                            # fallback: tentar ancestor <a> ou atributos do próprio element
                            try:
                                link = elemento.find_element(By.XPATH, ".//ancestor::a[1]").get_attribute('href')
                            except:
                                link = elemento.get_attribute('href') or elemento.get_attribute('data-url') or ''
                    except Exception:
                        link = ''
                    lista_ofertas.append((nome, preco, link))
            except:
                continue
    return(lista_ofertas)


### Contrução da nossa lista de ofertas encontradas

In [3]:
tabela_ofertas = pd.DataFrame()

for linha in tabela_produtos.index:
    produto = tabela_produtos.loc[linha, "Nome"]
    termos_banidos = tabela_produtos.loc[linha, "Termos banidos"]
    preco_minimo = tabela_produtos.loc[linha, "Preço mínimo"]
    preco_maximo = tabela_produtos.loc[linha, "Preço máximo"]

    lista_ofertas_google_shopping = busca_google_shopping(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_google_shopping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shopping, columns=['produto', 'preço', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping], ignore_index=True)
    else:
        tabela_google_shopping = None
    
    lista_ofertas_buscape = busca_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preço', 'link'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape], ignore_index=True)    
    else:
        tabela_buscape = None

display(tabela_ofertas)



busca realizada
apple iphone 13 128gb meia-noite está dentro do intervalo. Preço: 3419.1
vitirne iphone 13 128gb preto 5g está dentro do intervalo. Preço: 2563.1
apple iphone 13 128gb desbloqueado novo está dentro do intervalo. Preço: 3207.57
iphone 13 apple preto 5g 128gb (vitrine) está dentro do intervalo. Preço: 2706.55
iphone 13 apple tela 6,1 polegadas 5g ios 15 câmera 12mp 128gb - meia noite está dentro do intervalo. Preço: 3375.9
iphone 13 apple 128gb com acessorios - vitrine está dentro do intervalo. Preço: 2563.1
vitrine iphone 13 apple 128gb black câmera dupla 12mp está dentro do intervalo. Preço: 2563.1
vitrine iphone 13 rosa 5g 128gb está dentro do intervalo. Preço: 2563.1
iphone 13 apple 128gb camera 12mpx - vitrine está dentro do intervalo. Preço: 2563.1
vitrine iphone 13 pink 5g tela 6.1 oled 128gb - apple está dentro do intervalo. Preço: 2528.67
apple iphone 13 128gb estelar tela 6,1” 12mp está dentro do intervalo. Preço: 3499.0
iphone 13 black 5g apple 128gb está dentr

,produto,preço,link
0,apple iphone 13 128gb meia-noite,3419.10,https://www.magazineluiza.com.br/apple-iphone-...
1,vitirne iphone 13 128gb preto 5g,2563.10,https://www.carrefour.com.br/vitirne-iphone-13...
2,apple iphone 13 128gb desbloqueado novo,3207.57,https://www.carrefour.com.br/apple-iphone-13-1...
3,iphone 13 apple preto 5g 128gb (vitrine),2706.55,https://www.carrefour.com.br/iphone-13-apple-p...
4,"iphone 13 apple tela 6,1 polegadas 5g ios 15 c...",3375.90,https://www.buscape.com.br/offer?oid=109467610...
5,iphone 13 apple 128gb com acessorios - vitrine,2563.10,https://www.carrefour.com.br/iphone-13-apple-1...
6,vitrine iphone 13 apple 128gb black câmera dup...,2563.10,https://www.carrefour.com.br/vitrine-iphone-13...
7,vitrine iphone 13 rosa 5g 128gb,2563.10,https://www.carrefour.com.br/vitrine-iphone-13...
8,iphone 13 apple 128gb camera 12mpx - vitrine,2563.10,https://www.carrefour.com.br/iphone-13-apple-1...
9,vitrine iphone 13 pink 5g tela 6.1 oled 128gb ...,2528.67,https://www.carrefour.com.br/vitrine-iphone-13...


### Exportar a tabela de ofertas para um excel

In [4]:
tabela_ofertas.to_excel('ofertas.xlsx', index=False)

### Enviar email

In [ ]:
import smtplib
import email.message
import os
from dotenv import load_dotenv

# Carregar variáveis do .env
load_dotenv()

if len(tabela_ofertas.index) > 0:
    def enviar_email():
        msg = email.message.Message()
        msg["Subject"] = "Email enviado com Python"
        msg["From"] = os.getenv("EMAIL_FROM")
        msg["To"] = os.getenv("EMAIL_TO")
        #msg["Cc"] = "seuemailcopia@gmail.com;outroemailcopia@hotmail.com"
        #msg["Bcc"] = "seuemailcopiaoculta@gmail.com"
        
        #link_imagem = "coloque_aqui_o_link_da_sua_imagem"

        corpo_email = f"""<p>Prezados,</p>
        <p>Encontramos alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
        {tabela_ofertas.to_html(index=False)}
        <p>Qualquer dúvida estou à disposição</p>
        <p>Att.,</p>"""

        corpo_email = corpo_email.encode("utf-8")

        msg.add_header("Content-Type", "text/html")
        msg.set_payload(corpo_email)

        servidor = smtplib.SMTP("smtp.gmail.com", 587)
        servidor.starttls()
        servidor.login(msg["From"], os.getenv("senha_app"))
        servidor.send_message(msg)
        servidor.quit()
        print("Email enviado")


    enviar_email()

Email enviado
